# Single Chromosome Simulation

In [1]:
from OpenMiChroM.ChromDynamics import MiChroM
from OpenMiChroM.CndbTools import cndbTools

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

from openmm.app import *

In [2]:
block = 3*10**2
n_blocks = 2*10**3

**{ chr 10 } simulation**  

In [ ]:
sim = MiChroM(temperature=1.0, time_step=0.01)
sim.setup(platform="cuda") # Double-check CUDA installation in your system 
sim.saveFolder('single_op_chr10_2000')
Chrom10 = sim.createSpringSpiral(ChromSeq='../inputs/chr10_beads.txt', isRing=False)
sim.loadStructure(Chrom10, center=True)
sim.saveStructure(mode='pdb')

In [ ]:
sim.addFENEBonds(kfb=30.0)
sim.addAngles(ka=2.0)
sim.addRepulsiveSoftCore(Ecut=4.0)
sim.addTypetoType(mu=3.22, rc = 1.78)
sim.addIdealChromosome(mu=3.22, rc = 1.78, dinit=3, dend=500)
sim.addFlatBottomHarmonic( kr=5*10**-3, n_rad=15.0)

**collapse simulation**  

In [ ]:
print('GENERATING INITIAL SIM STRUCTURE', '\n---\n')
for _ in range(n_blocks):
    sim.runSimBlock(block, increment=False) 
    print('1st loop', '\n---\n')

In [ ]:
print(sim.chromRG())
sim.saveStructure(mode='pdb')

In [ ]:
# sim.system.getForces()      # WHY GET FORCES ??
sim.system.removeForce(5)
sim.initStorage(filename="traj_chr10_2000")

In [ ]:
pdb = PDBFile('single_op_chr10_2000/chromosome_0_block0.pdb')
top = pdb.getTopology()
top_file = 'single_top_2000.dcd'

**production simulation**  

In [ ]:
with open(top_file, 'wb') as f:
    dcd = DCDFile(f, top, 0.01)
    print('RUNNING SIMULATION', '\n---\n')
    for _ in range(n_blocks):
        sim.runSimBlock(block, increment=True) 
        sim.saveStructure()
        sim.saveStructure(mode='pdb')
        dcd.writeModel(sim.getPositions())
        print('2nd loop', '\n---\n')

In [ ]:
sim.storage[0].close()